In [4]:
# вимикаємо зайві попередження
import warnings
warnings.filterwarnings("ignore")

# друк всіх результатів в одній комірці а не тільки останнього
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.value = None


class Trie:
    def __init__(self):
        self.root = TrieNode()
        self.size = 0

    def put(self, key, value=None):
        if not isinstance(key, str) or not key:
            raise TypeError(
                f"Illegal argument for put: key = {key} must be a non-empty string"
            )

        current = self.root
        for char in key:
            if char not in current.children:
                current.children[char] = TrieNode()
            current = current.children[char]
        if current.value is None:
            self.size += 1
        current.value = value

    def get(self, key):
        if not isinstance(key, str) or not key:
            raise TypeError(
                f"Illegal argument for get: key = {key} must be a non-empty string"
            )

        current = self.root
        for char in key:
            if char not in current.children:
                return None
            current = current.children[char]
        return current.value

    def delete(self, key):
        if not isinstance(key, str) or not key:
            raise TypeError(
                f"Illegal argument for delete: key = {key} must be a non-empty string"
            )

        def _delete(node, key, depth):
            if depth == len(key):
                if node.value is not None:
                    node.value = None
                    self.size -= 1
                    return len(node.children) == 0
                return False

            char = key[depth]
            if char in node.children:
                should_delete = _delete(node.children[char], key, depth + 1)
                if should_delete:
                    del node.children[char]
                    return len(node.children) == 0 and node.value is None
            return False

        return _delete(self.root, key, 0)

    def is_empty(self):
        return self.size == 0

    def longest_prefix_of(self, s):
        if not isinstance(s, str) or not s:
            raise TypeError(
                f"Illegal argument for longestPrefixOf: s = {s} must be a non-empty string"
            )

        current = self.root
        longest_prefix = ""
        current_prefix = ""
        for char in s:
            if char in current.children:
                current = current.children[char]
                current_prefix += char
                if current.value is not None:
                    longest_prefix = current_prefix
            else:
                break
        return longest_prefix

    def keys_with_prefix(self, prefix):
        if not isinstance(prefix, str):
            raise TypeError(
                f"Illegal argument for keysWithPrefix: prefix = {prefix} must be a string"
            )

        current = self.root
        for char in prefix:
            if char not in current.children:
                return []
            current = current.children[char]

        result = []
        self._collect(current, list(prefix), result)
        return result

    def _collect(self, node, path, result):
        if node.value is not None:
            result.append("".join(path))
        for char, next_node in node.children.items():
            path.append(char)
            self._collect(next_node, path, result)
            path.pop()

    def keys(self):
        result = []
        self._collect(self.root, [], result)
        return result

In [ ]:
class Homework(Trie):
    def count_words_with_suffix(self, pattern) -> int:
        """
        Метод для підрахунку кількості слів, що закінчуються на заданий суфікс.
        :param pattern: суфікс, який потрібно знайти
        :return: кількість слів, що закінчуються на заданий суфікс
        
        Принципова реалізація:
        1. Так як суфікс за визначенням знаходиться в кінці слова, то:
            а) нам треба перебрати всі слова в дереві,
            б) при чому врахувати те, що недостатньо, щоб заданий рядок був містився в середині слова, нам треба щоб він був в кінці слова.
        2. Цей функціонал вже реалізовано в методі get, тому нам треба максимально його використати.
        3. Для пошуку входження суфікса вважаємо що суфікс - це таке слово і ми шукаємо його в дереві використовуючи вже існуючий метод get.
        4. Щоб використати метод get, нам потрібно застосувати його до кожної ноди, перевіряючи, чи не є я ця нода початком суфікса.
        5. Для цього ми рекурсивно приєднуємо кожну ноду до корня тимчасового дерева і перевіряємо, чи є суфікс (як окреме слово) в цьому дереві, використовуючи метод get() об'єкту Trie().
            Якщо так, то ми знайшли слово, що закінчується на суфікс, і збільшуємо лічильник.
        6. І так, спускаючись по нодам дерева до кінця слів, перевіряємо всі ноди, а потім повертаємо значення лічильника.
        7. Якщо я не помиляюся весь алгоритм працює за O(n*l), де n - кількість нод в дереві, l - довжина суфікса.
            Мене бентежить те, що для кожної ноди ми спускаємося по дереву на глибину суфікса при використанні метода get,
            а потім ще раз спускаємося вже для рекурсивного обходу нод. Як це оптимізувати я швидко не придумав.
            Тим не менш, обмежень щодо складності в завданні не було (як мінімум тут не квадратична чи гірше складність), тому я залишив так.
        """
        if not isinstance(pattern, str) or not pattern:
            raise TypeError(
                f"Illegal argument for count_words_with_suffix: prefix = {pattern} must be a non-empty string"
            )

        counter = 0
        def _count(node, pattern):
            nonlocal counter

            temp_trie = Trie()
            temp_trie.root = node

            if temp_trie.get(pattern) is not None:
                counter += 1

            if node.children:
                for key in node.children:
                    _count(node.children[key], pattern)

        _count(self.root, pattern)

        return counter

    def has_prefix(self, prefix=None) -> bool:
        '''
        Метод для перевірки наявності префікса в словнику.
        :param prefix: префікс, який потрібно знайти (значення по замовчанню = None, 
            щоб ця помилка (пусте значення) ловилася перевіркою в середині методу, як того вимагають умови завдання).
            Під префіксом розуміється будь-яка частина слова, що стоїть на початку слова, але яка не є повним словом.
        :return: True, якщо префікс знайдено, False - якщо не знайдено.
        '''
        if not isinstance(prefix, str) or not prefix:
            raise TypeError(
                f"Illegal argument for count_words_with_suffix: prefix = {prefix} must be a non-empty string"
            )
        
        current_node = self.root
        for char in prefix:
            if char in current_node.children:
                current_node = current_node.children[char]
            else:
                return False

        # Якщо ми дійшли до кінця префікса, то перевіряємо чи не є знайдений префікс повним словом.
        # Якщо це так, то повертаємо False, бо префікс != повне слово.
        if current_node.children:
            return True
        else:
            return False

    def contains(self, key):
        if not isinstance(key, str) or not key:
            raise TypeError(
                f"Illegal argument for contains: key = {key} must be a non-empty string"
            )
        return self.get(key) is not None


if __name__ == "__main__":
    trie = Homework()
    words = ["apple", "application", "banana", "cat", ]
    for i, word in enumerate(words):
        trie.put(word, i)

    # Перевірка кількості слів, що закінчуються на заданий суфікс
    assert trie.count_words_with_suffix("e") == 1  # apple
    assert trie.count_words_with_suffix("ion") == 1  # application
    assert trie.count_words_with_suffix("a") == 1  # banana
    assert trie.count_words_with_suffix("at") == 1  # cat
    # print(trie.count_words_with_suffix('le'))

    # Перевірка наявності префікса
    assert trie.has_prefix("app") == True  # apple, application
    assert trie.has_prefix("bat") == False
    assert trie.has_prefix("ban") == True  # banana
    assert trie.has_prefix("ca") == True  # cat
    # print(trie.has_prefix("banana"))